In [2]:
%pip install yfinance

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 0.0/3.0 MB ? eta -:--:--
     ------------- -------------------------- 1.0/3.0 MB 7.7 MB/s eta 0:00:01
     ---------------------------------------- 3.0/3.0 MB 10.1 MB/s  0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
   ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
   ---------------------------------------- 1.6/1.6 MB 11.9 MB/s  0:00:00
  Created wheel for multitasking: filename=multitasking-0.0.12-py3-none-any.whl size=15703 sha256=2c4d6b8e6572c91d158b8f0f18fbcafa387f25b74c4e4c26de0bb94821351588
  Stored in directory: c:\users\wongb

  DEPRECATION: Building 'multitasking' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'multitasking'. Discussion can be found at https://github.com/pypa/pip/issues/6334

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [62]:
my_tickers = ["VTI", "QQQM", "VUG", "SPMO", "XLV", "QUAL", "SCHD", "USMV", "VDC", "XLP", 'QQQ', 'VXUS', 'IEMG', 'VEA', 'AVDV', 'AVUV', 'VSS']

In [63]:
import yfinance as yf
import pandas as pd
import numpy as np

def analyze_recovery_time(
    tickers,
    start_date="1990-01-01",
    end_date=None,
    sort_by="Annualized Drawdown Area (%-days/yr)",
    ascending=False,
    add_rank=True,
    use_adjusted=True,
):
    # 1. Fetch price data
    raw_data = yf.download(
        tickers,
        start=start_date,
        end=end_date,
        progress=False,
        auto_adjust=False,  # keep both Close and Adj Close so we can choose
    )

    # Select price field based on flag and availability
    price_field = "Adj Close" if use_adjusted and "Adj Close" in raw_data.columns.get_level_values(0) else "Close"

    if len(tickers) > 1:
        data = raw_data[price_field]
    else:
        data = pd.DataFrame(raw_data[price_field])
    
    results = {}

    for ticker in tickers:
        if ticker not in data.columns:
            continue
        series = data[ticker].dropna()
        
        if len(series) == 0:
            continue
        
        # 2. Calculate the Running Maximum (the "high-water mark")
        running_max = series.cummax()
        
        # 3. Calculate Drawdown (Percentage below the high-water mark)
        drawdown = (series / running_max) - 1
        
        # 4. Identify "Underwater" periods (below threshold)
        is_underwater = drawdown < 0
        
        # 5. Underwater duration stats
        underwater_groups = (is_underwater != is_underwater.shift()).cumsum()
        underwater_only = underwater_groups[is_underwater]
        
        if not underwater_only.empty:
            durations = underwater_only.value_counts()
            max_recovery_days = durations.max()
            avg_recovery_days = durations.mean()
            p95_recovery_days = float(durations.quantile(0.95))
        else:
            max_recovery_days, avg_recovery_days, p95_recovery_days = 0, 0, 0
        
        total_days = len(series)
        underwater_ratio = round(is_underwater.sum() / total_days * 100, 2) if total_days else 0
        
        # 6. Drawdown area: sum of drawdown magnitudes across all underwater days
        drawdown_area = float((-drawdown[drawdown < 0]).sum() * 100)
        annualized_drawdown_area = round(drawdown_area / (total_days / 252), 2) if total_days else 0

        # 7. Ulcer Index (percent units) and ratios
        dd_neg = np.minimum(drawdown, 0)
        ulcer_index = float(np.sqrt((dd_neg ** 2).mean()) * 100) if len(dd_neg) else 0

        years = total_days / 252 if total_days else 0
        cagr = (series.iloc[-1] / series.iloc[0]) ** (1 / years) - 1 if years > 0 else np.nan
        max_drawdown_abs = abs(drawdown.min()) if len(drawdown) else np.nan

        calmar_ratio = cagr / max_drawdown_abs if max_drawdown_abs not in (0, np.nan) else np.nan
        martin_ratio = cagr / (ulcer_index / 100) if ulcer_index not in (0, np.nan) else np.nan
            
        results[ticker] = {
            "Max Drawdown (%)": drawdown.min() * 100,
            "Max Recovery (Trading Days)": max_recovery_days,
            "Avg Recovery (Trading Days)": round(avg_recovery_days, 1),
            "P95 Recovery (Trading Days)": round(p95_recovery_days, 1),
            "Underwater Days (%)": underwater_ratio,
            "Drawdown Area (%-days)": round(drawdown_area, 2),
            "Annualized Drawdown Area (%-days/yr)": annualized_drawdown_area,
            "Ulcer Index (%)": round(ulcer_index, 2),
            "Calmar Ratio": round(calmar_ratio, 3) if not pd.isna(calmar_ratio) else np.nan,
            "Martin Ratio": round(martin_ratio, 3) if not pd.isna(martin_ratio) else np.nan,
            "Current Status": "Underwater" if drawdown.iloc[-1] < 0 else "At High"
        }

    df = pd.DataFrame(results).T
    # 8. Sort and rank
    sort_col = sort_by if sort_by in df.columns else ("Max Drawdown (%)" if "Max Drawdown (%)" in df.columns else None)
    if sort_col is not None:
        df = df.sort_values(by=sort_col, ascending=ascending)
        if add_rank:
            df.insert(0, "Rank", df[sort_col].rank(ascending=not ascending, method="dense").astype(int))

    return df

# Full modern time scale
recovery_stats = analyze_recovery_time(my_tickers, start_date="1990-01-01", end_date=None)
from IPython.display import display
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 200)
display(recovery_stats.T)

,QQQ,VEA,IEMG,VSS,QQQM,VXUS,AVUV,VTI,XLV,VUG,XLP,AVDV,SPMO,QUAL,VDC,SCHD,USMV
Rank,17,16,15,14,13,12,11,10,9,8,7,6,5,4,3,2,1
Max Drawdown (%),-82.963909,-60.692344,-38.706516,-43.512007,-35.043805,-35.972353,-49.423467,-55.453729,-39.170246,-50.675226,-35.889073,-43.00608,-30.947557,-34.056718,-34.240713,-33.367066,-33.098904
Max Recovery (Trading Days),3747,1651,1108,932,516,711,308,1114,1365,823,1463,579,487,489,569,492,503
Avg Recovery (Trading Days),31.6,41.8,51.4,32.0,17.7,34.3,31.7,18.0,27.2,17.0,26.3,17.8,15.1,13.1,16.8,14.8,12.5
P95 Recovery (Trading Days),57.4,95.2,343.8,52.0,39.0,85.8,199.1,55.0,89.1,68.8,111.2,44.9,59.7,52.8,61.6,57.8,42.9
Underwater Days (%),93.32,95.73,96.31,93.58,88.2,94.34,95.03,89.54,92.99,88.37,93.05,89.67,84.25,85.97,89.73,86.87,85.41
Drawdown Area (%-days),223999.86,62797.34,44015.05,41323.54,11339.24,31423.56,13149.38,48102.19,48777.24,39278.11,48154.63,11040.42,11972.47,13130.27,20474.25,12047.25,10361.65
Annualized Drawdown Area (%-days/yr),8375.07,3415.7,3352.01,2475.88,2189.65,2112.79,2111.95,1965.58,1809.76,1796.39,1786.66,1773.22,1176.24,1057.81,936.39,851.83,732.64
Ulcer Index (%),43.9,18.39,16.01,12.62,13.27,10.81,11.88,12.74,9.96,12.15,10.31,10.49,7.16,7.18,6.11,5.16,4.83
Calmar Ratio,0.126,0.076,0.133,0.213,0.463,0.168,0.29,0.173,0.219,0.241,0.183,0.33,0.581,0.395,0.27,0.371,0.359


In [29]:
# Debug: Check the structure of downloaded data
test_data = yf.download(["VTI", "QQQM"], start="2018-01-01", progress=False)
print("Columns:", test_data.columns)
print("Column names level 0:", test_data.columns.get_level_values(0).unique() if isinstance(test_data.columns, pd.MultiIndex) else test_data.columns)
print("Data shape:", test_data.shape)
print(test_data.head())

Columns: MultiIndex([( 'Close', 'QQQM'),
            ( 'Close',  'VTI'),
            (  'High', 'QQQM'),
            (  'High',  'VTI'),
            (   'Low', 'QQQM'),
            (   'Low',  'VTI'),
            (  'Open', 'QQQM'),
            (  'Open',  'VTI'),
            ('Volume', 'QQQM'),
            ('Volume',  'VTI')],
           names=['Price', 'Ticker'])
Column names level 0: Index(['Close', 'High', 'Low', 'Open', 'Volume'], dtype='object', name='Price')
Data shape: (2005, 10)
Price      Close             High              Low             Open             Volume         
Ticker      QQQM         VTI QQQM         VTI QQQM         VTI QQQM         VTI   QQQM      VTI
Date                                                                                           
2018-01-02   NaN  121.840874  NaN  121.849684  NaN  121.153305  NaN  121.426568    NaN  3699700
2018-01-03   NaN  122.546104  NaN  122.616626  NaN  121.911424  NaN  121.990755    NaN  3052300
2018-01-04   NaN  123.01330

In [64]:
# DotCom Bubble Crash
recovery_stats = analyze_recovery_time(my_tickers, start_date="1999-01-01", end_date="2007-07-01")
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 200)
display(recovery_stats.T)

$AVDV: possibly delisted; no price data found  (1d 1999-01-01 -> 2007-07-01) (Yahoo error = "Data doesn't exist for startDate = 915166800, endDate = 1183262400")
$VEA: possibly delisted; no price data found  (1d 1999-01-01 -> 2007-07-01) (Yahoo error = "Data doesn't exist for startDate = 915166800, endDate = 1183262400")
$IEMG: possibly delisted; no price data found  (1d 1999-01-01 -> 2007-07-01) (Yahoo error = "Data doesn't exist for startDate = 915166800, endDate = 1183262400")
$SCHD: possibly delisted; no price data found  (1d 1999-01-01 -> 2007-07-01) (Yahoo error = "Data doesn't exist for startDate = 915166800, endDate = 1183262400")
$QUAL: possibly delisted; no price data found  (1d 1999-01-01 -> 2007-07-01) (Yahoo error = "Data doesn't exist for startDate = 915166800, endDate = 1183262400")
$VXUS: possibly delisted; no price data found  (1d 1999-01-01 -> 2007-07-01) (Yahoo error = "Data doesn't exist for startDate = 915166800, endDate = 1183262400")
$USMV: possibly delisted; no 

,QQQ,XLP,XLV,VTI,VUG,VDC
Rank,6,5,4,3,2,1
Max Drawdown (%),-82.963896,-35.889096,-33.268825,-34.659923,-11.079149,-8.156164
Max Recovery (Trading Days),1824,1463,1365,635,199,127
Avg Recovery (Trading Days),56.1,77.2,51.4,25.7,18.3,11.1
P95 Recovery (Trading Days),29.8,306.1,112.9,82.0,102.3,38.8
Underwater Days (%),96.7,97.66,96.39,91.63,89.53,85.23
Drawdown Area (%-days),117439.05,28707.9,18719.54,11365.57,2292.49,1336.0
Annualized Drawdown Area (%-days/yr),14160.12,3388.47,2209.52,1888.02,671.75,391.48
Ulcer Index (%),61.26,15.72,10.96,11.44,3.63,2.27
Calmar Ratio,-0.008,0.047,0.139,0.194,0.729,1.261


In [65]:
# GFC
recovery_stats = analyze_recovery_time(my_tickers, start_date="2007-01-01", end_date="2013-07-01")
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 200)
display(recovery_stats.T)

$AVDV: possibly delisted; no price data found  (1d 2007-01-01 -> 2013-07-01) (Yahoo error = "Data doesn't exist for startDate = 1167627600, endDate = 1372651200")
$AVUV: possibly delisted; no price data found  (1d 2007-01-01 -> 2013-07-01) (Yahoo error = "Data doesn't exist for startDate = 1167627600, endDate = 1372651200")
$QUAL: possibly delisted; no price data found  (1d 2007-01-01 -> 2013-07-01) (Yahoo error = "Data doesn't exist for startDate = 1167627600, endDate = 1372651200")
$SPMO: possibly delisted; no price data found  (1d 2007-01-01 -> 2013-07-01) (Yahoo error = "Data doesn't exist for startDate = 1167627600, endDate = 1372651200")
$QQQM: possibly delisted; no price data found  (1d 2007-01-01 -> 2013-07-01) (Yahoo error = "Data doesn't exist for startDate = 1167627600, endDate = 1372651200")

5 Failed downloads:
['AVDV', 'AVUV', 'QUAL', 'SPMO', 'QQQM']: possibly delisted; no price data found  (1d 2007-01-01 -> 2013-07-01) (Yahoo error = "Data doesn't exist for startDate = 1

,VEA,VTI,QQQ,VUG,VXUS,XLV,VSS,VDC,XLP,IEMG,SCHD,USMV
Rank,12,11,10,9,8,7,6,5,4,3,2,1
Max Drawdown (%),-60.692371,-55.453772,-53.404019,-50.675216,-26.984136,-39.17027,-29.091256,-34.240734,-32.38756,-17.49909,-6.838249,-6.643354
Max Recovery (Trading Days),1424,1114,781,823,507,849,544,569,378,123,63,63
Avg Recovery (Trading Days),122.9,36.7,27.8,27.2,59.0,28.0,21.0,16.4,15.7,25.3,6.5,6.0
P95 Recovery (Trading Days),656.2,71.3,103.7,83.0,295.0,80.6,51.8,60.1,71.9,97.0,26.6,28.0
Underwater Days (%),98.79,94.43,91.92,93.27,97.04,92.66,90.81,88.37,88.37,89.94,75.47,75.0
Drawdown Area (%-days),39579.62,23493.74,20395.4,19372.39,6817.98,15980.46,9719.13,8633.04,8141.77,666.5,528.77,497.91
Annualized Drawdown Area (%-days/yr),6680.55,3623.27,3145.44,2987.66,2825.87,2464.55,2297.58,1331.41,1255.65,993.83,314.27,295.93
Ulcer Index (%),29.23,19.83,18.29,17.36,13.31,13.57,12.13,8.86,8.33,5.43,1.95,1.85
Calmar Ratio,-0.029,0.085,0.165,0.12,0.001,0.195,0.568,0.285,0.292,-0.284,3.021,2.744


In [66]:
# 2011 Debt Ceiling Crisis
recovery_stats = analyze_recovery_time(my_tickers, start_date="2011-01-01", end_date="2011-12-01")
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 200)
display(recovery_stats.T)

$IEMG: possibly delisted; no price data found  (1d 2011-01-01 -> 2011-12-01) (Yahoo error = "Data doesn't exist for startDate = 1293858000, endDate = 1322715600")
$AVUV: possibly delisted; no price data found  (1d 2011-01-01 -> 2011-12-01) (Yahoo error = "Data doesn't exist for startDate = 1293858000, endDate = 1322715600")
$QQQM: possibly delisted; no price data found  (1d 2011-01-01 -> 2011-12-01) (Yahoo error = "Data doesn't exist for startDate = 1293858000, endDate = 1322715600")
$AVDV: possibly delisted; no price data found  (1d 2011-01-01 -> 2011-12-01) (Yahoo error = "Data doesn't exist for startDate = 1293858000, endDate = 1322715600")
$QUAL: possibly delisted; no price data found  (1d 2011-01-01 -> 2011-12-01) (Yahoo error = "Data doesn't exist for startDate = 1293858000, endDate = 1322715600")
$SPMO: possibly delisted; no price data found  (1d 2011-01-01 -> 2011-12-01) (Yahoo error = "Data doesn't exist for startDate = 1293858000, endDate = 1322715600")

6 Failed downloads:
[

,VXUS,VSS,VEA,VTI,VUG,XLV,QQQ,XLP,VDC,SCHD,USMV
Rank,11,10,9,8,7,6,5,4,3,2,1
Max Drawdown (%),-26.984114,-29.091243,-25.914732,-20.30408,-18.456334,-17.334345,-16.099285,-11.940737,-11.166751,-6.83824,-5.381696
Max Recovery (Trading Days),149,149,148,149,102,136,89,135,135,23,12
Avg Recovery (Trading Days),21.8,17.7,23.4,19.0,19.0,13.3,19.1,15.2,12.9,12.5,5.5
P95 Recovery (Trading Days),97.8,78.0,105.6,90.0,74.0,53.4,68.5,63.6,48.9,21.9,11.2
Underwater Days (%),92.02,91.77,91.34,90.48,90.48,86.15,90.91,85.28,83.55,86.21,75.86
Drawdown Area (%-days),2049.97,2174.89,2098.17,1373.74,1191.93,1059.88,1020.29,619.02,586.59,64.0,41.45
Annualized Drawdown Area (%-days/yr),2425.32,2372.61,2288.91,1498.62,1300.29,1156.24,1113.05,675.29,639.91,556.14,360.21
Ulcer Index (%),12.68,13.15,12.15,8.16,7.02,6.5,5.8,3.68,3.52,2.98,2.2
Calmar Ratio,-0.512,-0.631,-0.465,-0.055,0.082,0.516,0.171,0.997,1.036,5.535,4.493


In [67]:
# 2015-2016 Stock Market Selloff
recovery_stats = analyze_recovery_time(my_tickers, start_date="2015-01-01", end_date="2016-12-31")
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 200)
display(recovery_stats.T)

$QQQM: possibly delisted; no price data found  (1d 2015-01-01 -> 2016-12-31) (Yahoo error = "Data doesn't exist for startDate = 1420088400, endDate = 1483160400")
$AVUV: possibly delisted; no price data found  (1d 2015-01-01 -> 2016-12-31) (Yahoo error = "Data doesn't exist for startDate = 1420088400, endDate = 1483160400")
$AVDV: possibly delisted; no price data found  (1d 2015-01-01 -> 2016-12-31) (Yahoo error = "Data doesn't exist for startDate = 1420088400, endDate = 1483160400")

3 Failed downloads:
['QQQM', 'AVUV', 'AVDV']: possibly delisted; no price data found  (1d 2015-01-01 -> 2016-12-31) (Yahoo error = "Data doesn't exist for startDate = 1420088400, endDate = 1483160400")


,IEMG,VXUS,VEA,VSS,XLV,QQQ,VUG,VTI,XLP,SCHD,VDC,QUAL,USMV,SPMO
Rank,14,13,12,11,10,9,8,7,6,5,4,3,2,1
Max Drawdown (%),-33.105599,-24.601363,-22.331829,-22.504959,-17.093666,-16.104421,-15.601719,-15.134906,-10.074765,-13.924412,-10.005306,-11.487614,-9.50033,-11.031643
Max Recovery (Trading Days),424,411,407,411,367,163,246,241,119,259,117,109,112,162
Avg Recovery (Trading Days),60.9,33.9,33.6,31.7,31.9,18.3,21.1,13.8,17.7,19.8,16.0,19.7,16.6,62.8
P95 Recovery (Trading Days),292.4,154.9,152.8,135.2,138.1,63.4,80.0,31.2,101.2,78.6,53.6,69.8,77.0,148.0
Underwater Days (%),96.63,94.05,93.25,94.25,95.04,90.87,92.26,90.08,91.27,90.28,91.87,90.08,89.09,81.23
Drawdown Area (%-days),7625.58,5044.65,4404.33,4193.99,2984.53,1774.02,1584.33,1495.18,1333.28,1302.71,1257.0,1198.56,1087.81,648.84
Annualized Drawdown Area (%-days/yr),3812.79,2522.32,2202.17,2096.99,1492.26,887.01,792.16,747.59,666.64,651.35,628.5,599.28,543.9,529.15
Ulcer Index (%),17.44,11.69,10.25,9.87,7.3,5.0,4.53,4.44,3.51,3.73,3.34,3.47,3.01,3.1
Calmar Ratio,-0.07,0.014,0.062,0.092,0.102,0.523,0.309,0.426,0.603,0.561,0.638,0.614,0.835,0.744


In [68]:
# 2018 Q4 Correction
recovery_stats = analyze_recovery_time(my_tickers, start_date="2018-08-15", end_date="2019-01-15")
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 200)
display(recovery_stats.T)

$QQQM: possibly delisted; no price data found  (1d 2018-08-15 -> 2019-01-15) (Yahoo error = "Data doesn't exist for startDate = 1534305600, endDate = 1547528400")
$AVDV: possibly delisted; no price data found  (1d 2018-08-15 -> 2019-01-15) (Yahoo error = "Data doesn't exist for startDate = 1534305600, endDate = 1547528400")
$AVUV: possibly delisted; no price data found  (1d 2018-08-15 -> 2019-01-15) (Yahoo error = "Data doesn't exist for startDate = 1534305600, endDate = 1547528400")

3 Failed downloads:
['QQQM', 'AVDV', 'AVUV']: possibly delisted; no price data found  (1d 2018-08-15 -> 2019-01-15) (Yahoo error = "Data doesn't exist for startDate = 1534305600, endDate = 1547528400")


,VSS,SPMO,QQQ,VUG,VEA,VXUS,IEMG,QUAL,VTI,SCHD,XLV,USMV,VDC,XLP
Rank,14,13,12,11,10,9,8,7,6,5,4,3,2,1
Max Drawdown (%),-19.62404,-23.391092,-22.796673,-22.336886,-17.117185,-15.673911,-13.799592,-20.470708,-20.046498,-17.077704,-15.441443,-12.702283,-13.695983,-13.453149
Max Recovery (Trading Days),93,71,93,71,74,93,93,77,78,77,42,77,42,42
Avg Recovery (Trading Days),31.7,17.8,31.7,30.3,18.2,32.0,31.7,22.5,31.0,12.4,14.3,14.8,18.2,23.2
P95 Recovery (Trading Days),83.8,59.6,83.8,65.8,62.0,83.9,83.8,66.8,71.6,54.8,38.5,58.8,38.8,39.6
Underwater Days (%),91.35,85.58,91.35,87.5,87.5,92.31,91.35,86.54,89.42,83.65,82.69,85.58,87.5,89.42
Drawdown Area (%-days),934.73,837.29,814.01,775.36,734.74,723.27,712.08,674.77,651.38,538.75,430.45,330.5,325.49,318.61
Annualized Drawdown Area (%-days/yr),2264.93,2028.82,1972.4,1878.75,1780.32,1752.54,1725.41,1635.03,1578.34,1305.43,1043.02,800.83,788.69,772.01
Ulcer Index (%),10.68,10.31,9.91,9.62,8.6,8.23,7.71,8.58,8.2,6.77,5.71,4.37,4.58,4.5
Calmar Ratio,-1.097,-0.824,-1.054,-0.99,-0.904,-0.84,-0.258,-0.906,-0.945,-0.683,-0.403,-0.66,-0.608,-0.528


In [69]:
# COVID-19 Pandemic
recovery_stats = analyze_recovery_time(my_tickers, start_date="2020-01-01", end_date="2021-01-01")
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 200)
display(recovery_stats.T)

,AVUV,AVDV,VSS,VEA,VXUS,IEMG,USMV,SCHD,VTI,QUAL,VUG,QQQ,SPMO,VDC,XLP,XLV,QQQM
Rank,17,16,15,14,13,12,11,10,9,8,7,6,5,4,3,2,1
Max Drawdown (%),-49.075874,-43.006084,-39.805999,-35.269217,-34.933194,-34.668104,-33.098876,-33.367056,-35.000289,-34.056698,-31.780736,-28.559374,-30.947531,-25.306902,-24.512255,-28.404313,-8.418021
Max Recovery (Trading Days),225,233,206,206,204,188,222,154,121,121,75,72,96,121,123,120,30
Avg Recovery (Trading Days),35.0,61.0,26.0,26.0,20.7,19.4,39.2,19.5,10.1,11.7,9.5,9.8,8.9,16.4,16.2,13.0,10.2
P95 Recovery (Trading Days),159.3,198.8,127.2,127.2,105.5,89.0,168.5,82.5,46.0,53.5,61.0,61.6,26.9,61.8,63.2,44.8,26.2
Underwater Days (%),96.84,96.44,92.49,92.49,90.12,92.09,92.89,92.49,84.19,87.75,79.05,77.08,80.63,90.51,89.72,87.35,73.21
Drawdown Area (%-days),4599.89,3883.25,2604.66,2385.07,2342.9,2268.83,2104.69,1822.19,1771.81,1644.69,1433.37,1359.87,1334.82,1242.84,1222.17,978.67,98.26
Annualized Drawdown Area (%-days/yr),4581.71,3867.9,2594.37,2375.64,2333.64,2259.87,2096.37,1814.99,1764.81,1638.19,1427.7,1354.5,1329.55,1237.92,1217.34,974.8,442.15
Ulcer Index (%),22.43,18.55,14.33,12.74,12.66,12.86,10.63,10.35,10.69,9.73,9.13,8.42,8.29,7.12,6.89,6.32,2.83
Calmar Ratio,0.143,0.093,0.27,0.249,0.269,0.445,0.169,0.44,0.571,0.469,1.198,1.602,0.871,0.463,0.447,0.456,4.003


In [70]:
# 2022 Downturn
recovery_stats = analyze_recovery_time(my_tickers, start_date="2021-06-01", end_date="2024-12-01")
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 200)
display(recovery_stats.T)

,IEMG,VSS,VUG,QQQ,QQQM,VXUS,VEA,QUAL,VTI,SPMO,AVDV,AVUV,USMV,XLV,XLP,SCHD,VDC
Rank,17,16,15,14,13,12,11,10,9,8,7,6,5,4,3,2,1
Max Drawdown (%),-35.928434,-33.934069,-35.612969,-35.118723,-35.043816,-29.438746,-29.710546,-28.23035,-25.363336,-22.736188,-28.074907,-20.56533,-17.931437,-16.061039,-16.302388,-16.844317,-16.545842
Max Recovery (Trading Days),880,814,543,493,516,675,627,489,492,487,579,308,503,433,486,492,469
Avg Recovery (Trading Days),880.0,217.0,23.2,24.1,24.1,78.5,50.2,14.7,15.9,16.8,35.3,38.7,16.2,27.2,33.2,20.3,30.2
P95 Recovery (Trading Days),880.0,699.7,46.5,54.8,54.8,366.0,168.6,30.4,32.0,35.0,59.4,118.6,28.8,70.0,61.4,46.3,58.5
Underwater Days (%),99.77,98.41,89.34,90.02,90.14,97.96,96.71,88.32,88.21,87.87,96.03,96.49,88.32,92.4,93.99,91.95,92.52
Drawdown Area (%-days),16038.97,12360.94,10958.87,9937.4,9899.08,8382.25,7531.87,6958.22,6811.59,6572.37,6522.9,5422.92,4477.32,4256.04,4055.07,3871.12,3710.78
Annualized Drawdown Area (%-days/yr),4582.56,3531.7,3131.1,2839.26,2828.31,2394.93,2151.96,1988.06,1946.17,1877.82,1863.69,1549.41,1279.24,1216.01,1158.59,1106.03,1060.22
Ulcer Index (%),19.89,16.19,16.83,15.74,15.7,11.91,11.12,11.16,10.64,9.8,9.7,7.58,6.78,6.0,5.95,5.77,5.58
Calmar Ratio,-0.102,-0.039,0.373,0.389,0.392,0.025,0.059,0.44,0.439,0.833,0.116,0.509,0.548,0.456,0.466,0.461,0.502


In [71]:
# 2025 Trump Trade War
recovery_stats = analyze_recovery_time(my_tickers, start_date="2025-01-01", end_date="2025-11-30")
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 200)
display(recovery_stats.T)

,AVUV,XLV,QQQ,QQQM,VUG,SCHD,VTI,QUAL,SPMO,XLP,VDC,IEMG,USMV,VSS,VEA,VXUS,AVDV
Rank,17,16,15,14,13,12,11,10,9,8,7,6,5,4,3,2,1
Max Drawdown (%),-25.078048,-13.636939,-22.768304,-22.703416,-22.845426,-14.017242,-19.303595,-17.960865,-20.127167,-8.533077,-8.909194,-14.658559,-9.359483,-12.982129,-13.453336,-13.579963,-14.174541
Max Recovery (Trading Days),150,173,85,85,85,189,88,105,60,115,120,33,54,27,26,30,26
Avg Recovery (Trading Days),31.0,20.8,10.2,10.2,9.2,43.6,10.3,13.3,7.4,27.0,36.0,6.7,11.3,5.2,5.4,5.4,5.2
P95 Recovery (Trading Days),120.0,100.5,30.6,30.6,24.2,155.0,31.9,47.6,23.5,99.2,107.5,20.0,38.7,19.1,12.9,12.6,12.0
Underwater Days (%),95.18,91.23,80.26,80.26,81.14,95.61,81.58,87.28,77.63,94.74,94.74,73.68,89.04,73.68,75.44,71.49,70.18
Drawdown Area (%-days),1613.31,1276.85,902.68,901.17,893.53,836.02,719.17,713.14,707.56,657.85,638.33,355.67,303.05,287.39,281.34,281.11,262.07
Annualized Drawdown Area (%-days/yr),1783.14,1411.25,997.7,996.03,987.58,924.02,794.88,788.2,782.04,727.09,705.52,393.11,334.95,317.64,310.96,310.7,289.66
Ulcer Index (%),9.18,6.99,6.47,6.46,6.48,4.68,5.2,4.89,5.25,3.57,3.43,2.71,1.93,2.23,2.26,2.28,2.25
Calmar Ratio,0.279,1.314,1.071,1.076,0.989,0.325,1.002,0.761,1.469,0.417,0.52,2.318,1.032,2.346,2.608,2.42,3.572
